In [1]:
import pandas as pd
import re 
import nltk as nltk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import time
import math
import string
import gensim
import pyLDAvis
import seaborn as sns 
import matplotlib.pyplot as plt
#import pyLDAvis.gensim
#import pyLDAvis.gensim_models as gensimvis

from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore

In [2]:
import pandarallel
import multiprocessing
from pandarallel import pandarallel

num_processors = multiprocessing.cpu_count()

pandarallel.initialize(nb_workers=num_processors, use_memory_fs=False)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [3]:
%%time 

input_path = r"C:\Users\verma\OneDrive - The University of Chicago\Desktop\MSCA\QUARTER 4 SUMMER\NLP\FINAL PROJECT\data.parquet"

df = pd.read_parquet(input_path, engine='pyarrow')

CPU times: total: 24.8 s
Wall time: 1min 21s


In [4]:
df.head()

,date,title,text,year,month,day,dayofweek,clean_title,clean_text
0,2020-02-27,Children With Autism Saw Their Learning and So...,\nChildren With Autism Saw Their Learning and ...,2020,2,27,3,Children With Autism Saw Their Learning and So...,Children With Autism Saw Their Learning and So...
1,2021-11-24,College of Technology to launch AI Retail Lab ...,\nCollege of Technology to launch AI Retail L...,2021,11,24,2,College of Technology to launch AI Retail Lab ...,College of Technology to launch AI Retail Lab ...
2,2022-06-09,Child actor Claude Jarman Jr. looks back on hi...,\nChild actor Claude Jarman Jr. looks back on ...,2022,6,9,3,Child actor Claude Jarman Jr. looks back on hi...,Child actor Claude Jarman Jr. looks back on hi...
3,2020-04-17,Cr Bard Inc Has Returned 48.9% Since SmarTrend...,\n\nCr Bard Inc Has Returned 48.9% Since SmarT...,2020,4,17,4,Cr Bard Inc Has Returned 48.9 Since SmarTrend ...,Cr Bard Inc Has Returned 48.9 Since SmarTrend ...
4,2021-02-25,MulticoreWare Inc. Becomes CEVA’s Trusted Part...,\n\nMulticoreWare Inc. Becomes CEVA’s Trusted ...,2021,2,25,3,MulticoreWare Inc. Becomes CEVAs Trusted Partn...,MulticoreWare Inc. Becomes CEVAs Trusted Partn...


In [6]:
df.shape

(199208, 9)

In [7]:
# Calculate the number of rows for 10% of the data
ten_percent = int(len(df) * 0.1)

# Randomly select 10% of the data
sampled_cleandf = df.sample(n=ten_percent, random_state=42)  # Use a fixed seed for reproducibility

In [8]:
sampled_cleandf.shape

(19920, 9)

In [9]:
sampled_cleandf.head(5)

,date,title,text,year,month,day,dayofweek,clean_title,clean_text
135881,2022-06-08,QYSEA's Innovative AI Vision Lock Platform Set...,QYSEA's Innovative AI Vision Lock Platform Set...,2022,6,8,2,QYSEAs Innovative AI Vision Lock Platform Sets...,QYSEAs Innovative AI Vision Lock Platform Sets...
87337,2023-05-25,PayPal Is Using AI to Supercharge Profits | Na...,\nPayPal Is Using AI to Supercharge Profits | ...,2023,5,25,3,PayPal Is Using AI to Supercharge Profits Nasdaq,PayPal Is Using AI to Supercharge Profits Nasd...
117475,2023-02-12,Professor Cath Ellis: ChatGPT And The Controve...,\n \nProfessor Cath Ellis: ChatGPT And The Con...,2023,2,12,6,Professor Cath Ellis: ChatGPT And The Controve...,Professor Cath Ellis: ChatGPT And The Controv...
54075,2023-02-24,Facebook Parent Meta Wants to Show It's Still ...,\n\n\nFacebook Parent Meta Wants to Show It's ...,2023,2,24,4,Facebook Parent Meta Wants to Show Its Still a...,Facebook Parent Meta Wants to Show Its Still a...
66928,2021-03-23,Artificial Intelligence Market is Expected to ...,\nArtificial Intelligence Market is Expected t...,2021,3,23,1,Artificial Intelligence Market is Expected to ...,Artificial Intelligence Market is Expected to ...


### NER will be applied to filter out the industries and the Countries that are most commonly used in the texts '

In [10]:
# !python -m spacy download en_core_web_lg


[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     -------------------------------------- 587.7/587.7 MB 2.5 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [10]:
%%time

import pandas as pd
import spacy

# Load the spaCy model
nlp = spacy.load('en_core_web_lg')

entities = []
labels = []
position_start = []
position_end = []

# Iterate over each text in the 'clean_text' column
for text in sampled_cleandf['clean_text']:
    # Apply spaCy NLP pipeline to extract entities
    doc = nlp(text)
    
    # Extract entity details and append to respective lists
    for ent in doc.ents:
        entities.append(ent.text)
        labels.append(ent.label_)
        position_start.append(ent.start_char)
        position_end.append(ent.end_char)
        
# Create a new DataFrame from the extracted entity details
entity_df = pd.DataFrame({'Entities': entities, 'Labels': labels, 'Position_Start': position_start, 'Position_End': position_end})

KeyboardInterrupt: 

In [37]:
# Extract top organizations
top_organizations = entity_df[entity_df['Labels'] == 'ORG']['Entities'].value_counts().head(10)

# Extract top locations
top_locations = entity_df[entity_df['Labels'] == 'GPE']['Entities'].value_counts().head(10)

In [38]:
# Print the top organizations
print("Top Organizations:")
print(top_organizations)

Top Organizations:
Entities
AI                         372
ChatGPT                    197
Microsoft                  157
Google                     127
Gray Media Group           105
Gray Television Inc         64
GPT                         42
OpenAI                      40
Facebook                    38
Artificial Intelligence     38
Name: count, dtype: int64


In [39]:
# Print the top locations
print("Top Locations:")
print(top_locations)

Top Locations:
Entities
US            220
China          79
PRNewswire     79
India          55
Japan          35
UK             30
Russia         24
New York       23
Canada         21
France         21
Name: count, dtype: int64
